![PANGAEA_Banner.png](https://gitlab.awi.de/kriemann/nfdi4earth_academy_data/raw/main/logo/PANGAEA_Banner.png)

# Cleaning and Preprocessing Data

based on [PANGAEA Community Workshop script](https://github.com/pangaea-data-publisher/community-workshop-material/tree/master/Python/Data_curation_checklist)  
Last updated: 2025-08-18

This notebook aims to help you, to check and resolve some common problems and issues with data tables, before submitting them to a data repository like [PANGAEA](https://www.pangaea.de/).

Check out our [instructions](https://wiki.pangaea.de/wiki/Data_submission) and [data templates](https://wiki.pangaea.de/wiki/Best_practice_manuals_and_templates) for submissions to PANGAEA.

## 1. Import libraries/packages

In [1]:
import os

import numpy as np
import pandas as pd
import re

### date/time conversion package
from datetime import date, timedelta

### needed to download PANGAEA parameters
import requests

### Latitude/Longitude conversion package
from LatLon23 import string2latlon

### needed to find matching parameters
from difflib import get_close_matches 

## 2. Random test data
We created a random number data file for a PANGAEA test submission. We implemented on purpose very common formatting errors.  
Please open the file *random_test_data.txt* and try to find and correct all errors.

In [2]:
### get current working directory
datapath = os.getcwd( )
datapath = datapath+'/data/'
print(datapath)

/isibhv/projects/p_pangaea_proces/kriemann/python/2025_informatica/data/


In [3]:
#df.to_csv((datapath+'random_test_data.txt'),index=False,sep='\t',encoding='utf-8')

In [4]:
### open test data as dataframe
df = pd.read_csv(datapath+'random_test_data.txt', sep='\t')

In [5]:
### show dataframe, but only the first 5 rows
df.head()

,date,time,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),Empty
0,2017-02-02,17:00:00,Comment,Comment,Comment,Comment,Comment,Comment,Comment,Comment,NaN
1,2014-07-24,23:00:00,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,-999.9,0.9956999240091375,NaN
2,2012-07-15,18:00:00,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,"69,4",12.0,NaN,0.9250612779786214,NaN
3,2013-03-03,02:00:00,"Bremerhaven, Germany",EX,Stenella coeruleoalba,53°32'59.9964''N; 8°34'59.9988''E,69.49999999999999,12.5,2.5,0.5707431649702995,NaN
4,2012-02-10,09:00:00,"Bremerhaven, Germany",LC,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.59999999999998,13.0,3.0,0.14828220296065653,NaN


In [6]:
### show dataframe, but only the last 5 rows
df.tail()

,date,time,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),Empty
194,2018-11-17,20:00:00,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,88.5999999999989,108.0,98.0,0.802745192796905,NaN
195,2020-05-19,02:00:00,"Bremerhaven, Germany",EX,Stenella coeruleoalba,53°32'59.9964''N; 8°34'59.9988''E,88.6999999999989,108.5,98.5,0.10935532817792304,NaN
196,2013-11-15,21:00:00,"Bremerhaven, Germany",LC,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,88.79999999999889,109.0,99.0,0.3668977429172354,NaN
197,2014-06-23,18:00:00,"Bremerhaven, Germany",NT,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,88.89999999999888,109.5,99.5,0.28897221922623695,NaN
198,2012-07-21,06:00:00,Total sum,Total sum,Total sum,Total sum,Total sum,Total sum,Total sum,Total sum,NaN


In [59]:
df

,Event [],DATE/TIME [],LATITUDE [],LONGITUDE [],Latitude 2 [],Longitude 2 [],"DEPTH, water [m]",Location [],Species [],IUCN Red List status [],"Temperature, water [°C]",Uniform resource locator/link to reference []
1,PS132_1-2,2014-07-24T23:00:00,69.3,11.5,53.549999,8.583333,NaN,"Bremerhaven, Germany",Tursiops truncatus,Near Threatened,0.995700,https://doi.org/10.1594/PANGAEA.945749
2,PS132_1-2,2012-07-15T18:00:00,69.4,12.0,53.549999,8.583333,NaN,"Bremerhaven, Germany",Delphinus delphis,Endangered,0.925061,https://doi.org/10.1594/PANGAEA.945749
3,PS132_1-2,2013-03-03T02:00:00,69.5,12.5,53.549999,8.583333,0.025,"Bremerhaven, Germany",Stenella coeruleoalba,Extinct in the Wild,0.570743,https://doi.org/10.1594/PANGAEA.945749
4,PS132_1-2,2012-02-10T09:00:00,69.6,13.0,53.549999,8.583333,0.030,"Bremerhaven, Germany",Tursiops truncatus,Least Concern,0.148282,https://doi.org/10.1594/PANGAEA.945749
5,PS132_1-2,2016-05-03T12:00:00,69.7,13.5,53.549999,8.583333,0.035,"Bremerhaven, Germany",Delphinus delphis,Near Threatened,0.912003,https://doi.org/10.1594/PANGAEA.945749
...,...,...,...,...,...,...,...,...,...,...,...,...
193,PS132_1-2,2019-05-23T04:00:00,88.5,107.5,53.549999,8.583333,0.975,"Bremerhaven, Germany",Tursiops truncatus,Near Threatened,0.065561,https://doi.org/10.1594/PANGAEA.945749
194,PS132_1-2,2018-11-17T20:00:00,88.6,108.0,53.549999,8.583333,0.980,"Bremerhaven, Germany",Delphinus delphis,Endangered,0.802745,https://doi.org/10.1594/PANGAEA.945749
195,PS132_1-2,2020-05-19T02:00:00,88.7,108.5,53.549999,8.583333,0.985,"Bremerhaven, Germany",Stenella coeruleoalba,Extinct in the Wild,0.109355,https://doi.org/10.1594/PANGAEA.945749
196,PS132_1-2,2013-11-15T21:00:00,88.8,109.0,53.549999,8.583333,0.990,"Bremerhaven, Germany",Tursiops truncatus,Least Concern,0.366898,https://doi.org/10.1594/PANGAEA.945749


## 3. Data Curation Checklist



### 3.1 Data structure

* One cell = one entry
* Columns = Variable/Parameters
* Rows = Single observation
* Single header row
* No mix of numeric values and strings



#### Check data structure

In [7]:
### Check structure of data table
df.head(2)

,date,time,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),Empty
0,2017-02-02,17:00:00,Comment,Comment,Comment,Comment,Comment,Comment,Comment,Comment,NaN
1,2014-07-24,23:00:00,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,-999.9,0.9956999240091375,NaN


In [8]:
### Alternative way to see head and tail together
pd.concat([df.head(4), df.tail(4)])

,date,time,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),Empty
0,2017-02-02,17:00:00,Comment,Comment,Comment,Comment,Comment,Comment,Comment,Comment,NaN
1,2014-07-24,23:00:00,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,-999.9,0.9956999240091375,NaN
2,2012-07-15,18:00:00,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,"69,4",12.0,NaN,0.9250612779786214,NaN
3,2013-03-03,02:00:00,"Bremerhaven, Germany",EX,Stenella coeruleoalba,53°32'59.9964''N; 8°34'59.9988''E,69.49999999999999,12.5,2.5,0.5707431649702995,NaN
195,2020-05-19,02:00:00,"Bremerhaven, Germany",EX,Stenella coeruleoalba,53°32'59.9964''N; 8°34'59.9988''E,88.6999999999989,108.5,98.5,0.10935532817792304,NaN
196,2013-11-15,21:00:00,"Bremerhaven, Germany",LC,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,88.79999999999889,109.0,99.0,0.3668977429172354,NaN
197,2014-06-23,18:00:00,"Bremerhaven, Germany",NT,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,88.89999999999888,109.5,99.5,0.28897221922623695,NaN
198,2012-07-21,06:00:00,Total sum,Total sum,Total sum,Total sum,Total sum,Total sum,Total sum,Total sum,NaN


#### Remove rows that are not observations
Data table should have:
* only single header row
* no rows with aggregated statistics

In [9]:
### Remove row with comments, which is first row here
df = df.iloc[1: , :] # [row, column]

In [10]:
pd.concat([df.head(2), df.tail(2)])

,date,time,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),Empty
1,2014-07-24,23:00:00,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,-999.9,0.9956999240091375,NaN
2,2012-07-15,18:00:00,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,"69,4",12.0,NaN,0.9250612779786214,NaN
197,2014-06-23,18:00:00,"Bremerhaven, Germany",NT,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,88.89999999999888,109.5,99.5,0.28897221922623695,NaN
198,2012-07-21,06:00:00,Total sum,Total sum,Total sum,Total sum,Total sum,Total sum,Total sum,Total sum,NaN


In [11]:
### Remove row with aggregated statistics, which is last row here
df = df.iloc[:-1 , :] # [row, column]
pd.concat([df.head(2), df.tail(2)])

,date,time,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),Empty
1,2014-07-24,23:00:00,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,-999.9,0.9956999240091375,NaN
2,2012-07-15,18:00:00,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,"69,4",12.0,NaN,0.9250612779786214,NaN
196,2013-11-15,21:00:00,"Bremerhaven, Germany",LC,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,88.79999999999889,109.0,99.0,0.3668977429172354,NaN
197,2014-06-23,18:00:00,"Bremerhaven, Germany",NT,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,88.89999999999888,109.5,99.5,0.28897221922623695,NaN


### 3.2. Data types

* Are data types as expected?

__Numeric data__
* Columns must contain numbers only; exception see [quality flags](https://wiki.pangaea.de/wiki/Quality_flag), DO NOT USE FORMULAS (Excel) - cells with formulas have to be saved as number prior to submission
* Fields without data should be left empty (and NOT filled with '-', 'n/a', 'NaN', -9999 or '*' etc.).

In [12]:
### Check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 1 to 197
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      197 non-null    object 
 1   time                      197 non-null    object 
 2   Location                  197 non-null    object 
 3   Threatened status         197 non-null    object 
 4   species                   197 non-null    object 
 5   lat/lon                   197 non-null    object 
 6   latitude (deg)            197 non-null    object 
 7   lngitude (deg)            197 non-null    object 
 8   water depth (cm)          196 non-null    object 
 9   water temperature (degC)  197 non-null    object 
 10  Empty                     0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 17.1+ KB


In [13]:
df.head(2)

,date,time,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),Empty
1,2014-07-24,23:00:00,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,-999.9,0.9956999240091375,NaN
2,2012-07-15,18:00:00,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,"69,4",12.0,NaN,0.9250612779786214,NaN


#### Remove all columns containing nothing

In [14]:
### Remove all columns containing nothing (just NaN)
df.dropna(axis = 1, how = 'all', inplace = True)

#### Remove ambigious missing values

In [15]:
### replace missing values with empty cells
df['water depth (cm)'] = df['water depth (cm)'].str.replace('-999.9', '', regex=False)
df['water depth (cm)'] = df['water depth (cm)'].str.replace('n/a', '', regex=False)

#### Check for comma separated values

In [16]:
### List unique values
df['latitude (deg)'].unique()[:10]

array(['69.3', '69,4', '69.49999999999999', '69.59999999999998',
       '69.69999999999997', '69.79999999999997', '69.89999999999996',
       '69.99999999999996', '70.09999999999995', '70.19999999999995'],
      dtype=object)

In [17]:
### Replace commas with dots
def replace_comma(dataframe, char = ",", new_char = "."):
  return [re.sub(char, new_char, x) if isinstance(x, str) else x for x in dataframe]

df['latitude (deg)'] = replace_comma(df['latitude (deg)'])

#### Convert to numeric

In [18]:
### Convert specific column to numeric
df['latitude (deg)'] = pd.to_numeric(df['latitude (deg)'])

In [19]:
### OR: Convert multiple columns of dataframe to numeric
df[df.columns[6:]] = df[df.columns[6:]].apply(pd.to_numeric)

In [20]:
### Check data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 1 to 197
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      197 non-null    object 
 1   time                      197 non-null    object 
 2   Location                  197 non-null    object 
 3   Threatened status         197 non-null    object 
 4   species                   197 non-null    object 
 5   lat/lon                   197 non-null    object 
 6   latitude (deg)            197 non-null    float64
 7   lngitude (deg)            197 non-null    float64
 8   water depth (cm)          195 non-null    float64
 9   water temperature (degC)  197 non-null    float64
dtypes: float64(4), object(6)
memory usage: 15.5+ KB


#### Remove leading and trailing white space

In [21]:
df.columns

Index(['date', 'time', 'Location', 'Threatened status', 'species', 'lat/lon',
       'latitude (deg)', 'lngitude (deg)', 'water depth (cm)',
       'water temperature (degC)'],
      dtype='object')

In [22]:
### Remove all leading and trailing white spaces 
cols = df.select_dtypes(['object']).columns
df[cols] = df[cols].apply(lambda x: x.str.strip())

### Optional: Remove double white spaces
df.replace(r"\s", r" ", regex = True, inplace = True)

### 3.3. Date formatting
Date/Time should be provided in the ISO-format (e.g. 1954-04-07T13:34:11) as coordinated universal time (UTC) <p>
--> for local time it is better to use new "local time" column in addition to column providing UTC time in ISO-format

In [23]:
df.head(2)

,date,time,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC)
1,2014-07-24,23:00:00,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,NaN,0.995700
2,2012-07-15,18:00:00,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,69.4,12.0,NaN,0.925061


#### Merge Date and Time and convert to PANGAEA format

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 1 to 197
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      197 non-null    object 
 1   time                      197 non-null    object 
 2   Location                  197 non-null    object 
 3   Threatened status         197 non-null    object 
 4   species                   197 non-null    object 
 5   lat/lon                   197 non-null    object 
 6   latitude (deg)            197 non-null    float64
 7   lngitude (deg)            197 non-null    float64
 8   water depth (cm)          195 non-null    float64
 9   water temperature (degC)  197 non-null    float64
dtypes: float64(4), object(6)
memory usage: 15.5+ KB


In [25]:
### Join date and time
df['date/time'] = pd.to_datetime(df['date'] + ' ' + df['time'])

df.head(2)

,date,time,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),date/time
1,2014-07-24,23:00:00,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,NaN,0.995700,2014-07-24 23:00:00
2,2012-07-15,18:00:00,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,69.4,12.0,NaN,0.925061,2012-07-15 18:00:00


In [26]:
### Convert to pangaea standard time format
df['date/time'] = df['date/time'].dt.strftime('%Y-%m-%dT%H:%M:%S')

df.head(2)

,date,time,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),date/time
1,2014-07-24,23:00:00,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,NaN,0.995700,2014-07-24T23:00:00
2,2012-07-15,18:00:00,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,69.4,12.0,NaN,0.925061,2012-07-15T18:00:00


In [27]:
### Remove redundant time column
df = df.drop(['date','time'], axis = 1)

df.head(2)

,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),date/time
1,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,NaN,0.995700,2014-07-24T23:00:00
2,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,69.4,12.0,NaN,0.925061,2012-07-15T18:00:00


#### Convert from degrees to decimal format
Note: decimal degree (-65.1234) (S and W are negative, projection WGS84)

##### First separate latitude and longitude into individual columns
Note: Multiple values separated by ';', '-', '±', '()' (ranges, values with errors, uncertainties, or alternative values in brackets) within a single cell should be avoided

In [28]:
df.head(2)

,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),date/time
1,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,NaN,0.995700,2014-07-24T23:00:00
2,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,69.4,12.0,NaN,0.925061,2012-07-15T18:00:00


In [29]:
### Split cell by ; seperator
df['lat'] = [x.split(";")[0] for x in  df['lat/lon']]
df['lon'] = [x.split(";")[1] for x in  df['lat/lon']]

df.head(2)

,Location,Threatened status,species,lat/lon,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),date/time,lat,lon
1,"Bremerhaven, Germany",NT,Tursiops truncatus,53°32'59.9964''N; 8°34'59.9988''E,69.3,11.5,NaN,0.995700,2014-07-24T23:00:00,53°32'59.9964''N,8°34'59.9988''E
2,"Bremerhaven, Germany",EN,Dolphinus delphis,53°32'59.9964''N; 8°34'59.9988''E,69.4,12.0,NaN,0.925061,2012-07-15T18:00:00,53°32'59.9964''N,8°34'59.9988''E


In [30]:
### Remove old lat/lon column
df = df.drop('lat/lon', axis = 1)

df.head(2)

,Location,Threatened status,species,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),date/time,lat,lon
1,"Bremerhaven, Germany",NT,Tursiops truncatus,69.3,11.5,NaN,0.995700,2014-07-24T23:00:00,53°32'59.9964''N,8°34'59.9988''E
2,"Bremerhaven, Germany",EN,Dolphinus delphis,69.4,12.0,NaN,0.925061,2012-07-15T18:00:00,53°32'59.9964''N,8°34'59.9988''E


Then convert from degrees to decimal format

In [31]:
### Function to convert latitude and longitude from degrees to decimal units
def geo_decimal(latitude, longitude, format = "d%°%m%'%S%''%H", decimals = 6):
    lat_dec = [round(string2latlon(lat, long, format).lat.decimal_degree, decimals) for lat, long in zip(latitude, longitude)]
    long_dec = [round(string2latlon(lat, long, format).lon.decimal_degree, decimals) for lat, long in zip(latitude, longitude)]

    return lat_dec, long_dec
    

In [32]:
### call geo_decimal function
df['lat'], df['lon'] =  geo_decimal(df['lat'], df['lon'], format = "d%°%m%'%S%''%H")

df.head(2)

,Location,Threatened status,species,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),date/time,lat,lon
1,"Bremerhaven, Germany",NT,Tursiops truncatus,69.3,11.5,NaN,0.995700,2014-07-24T23:00:00,53.549999,8.583333
2,"Bremerhaven, Germany",EN,Dolphinus delphis,69.4,12.0,NaN,0.925061,2012-07-15T18:00:00,53.549999,8.583333


### 3.4. Spelling

#### Spell out abbreviations

In [33]:
### What are the abbreviations
df['Threatened status'].unique()

array(['NT', 'EN', 'EX', 'LC'], dtype=object)

In [34]:
### Create list with abbreviations
abbreviated = ('LC', 'NT', 'EN', 'EX')

In [35]:
### Create list with full names
spelled_out = ('Least Concern', 'Near Threatened', 'Endangered', 'Extinct in the Wild')

In [36]:
### Replace the values in 'Name' column with the dictionary
df = df.replace({"Threatened status": dict(zip(abbreviated, spelled_out))})

df.head()

,Location,Threatened status,species,latitude (deg),lngitude (deg),water depth (cm),water temperature (degC),date/time,lat,lon
1,"Bremerhaven, Germany",Near Threatened,Tursiops truncatus,69.3,11.5,NaN,0.995700,2014-07-24T23:00:00,53.549999,8.583333
2,"Bremerhaven, Germany",Endangered,Dolphinus delphis,69.4,12.0,NaN,0.925061,2012-07-15T18:00:00,53.549999,8.583333
3,"Bremerhaven, Germany",Extinct in the Wild,Stenella coeruleoalba,69.5,12.5,2.5,0.570743,2013-03-03T02:00:00,53.549999,8.583333
4,"Bremerhaven, Germany",Least Concern,Tursiops truncatus,69.6,13.0,3.0,0.148282,2012-02-10T09:00:00,53.549999,8.583333
5,"Bremerhaven, Germany",Near Threatened,Dolphinus delphis,69.7,13.5,3.5,0.912003,2016-05-03T12:00:00,53.549999,8.583333


#### Correct species names

In [37]:
### Create data frame with unique species names
species = pd.DataFrame({'name': df["species"].drop_duplicates()})

In [38]:
### Save as csv for upload to WoRMS or ITIS
species.to_csv('Species.txt', index=False, sep="\t")
print(species)

                    name
1     Tursiops truncatus
2      Dolphinus delphis
3  Stenella coeruleoalba


--> Use the species.txt file to check species spelling using the taxon match tools of:
- [WoRMS](https://www.marinespecies.org/aphia.php?p=match) 
- [ITIS](https://www.itis.gov/taxmatch.html)

After this you can also add columns for the AphiaID (WoRMS) or TSN numbers (ITIS)

Alternatively use the R package [taxize](https://cran.r-project.org/web/packages/taxize/index.html)

For species spelling conventions read our [PANGAEA wiki](https://wiki.pangaea.de/wiki/Taxon)

##### Correct individual species names

In [39]:
### Correct mispelled species names
df['species'] = df['species'].str.replace('Dolphinus delphis', 'Delphinus delphis')

In [40]:
### check current spelling
df['species'].unique()

array(['Tursiops truncatus', 'Delphinus delphis', 'Stenella coeruleoalba'],
      dtype=object)

### 3.5 Parameter (header) naming

* Use clear and descriptive parameter names, no cryptic abbreviations. Non experts should be able to understand the meaning (e.g. "Water content, sediment [%]" instead of "WCS")
* All parameters in English only
* Parameters are always accompanied by a unit
* Use SI units written in square brackets following the parameter e.g. "Ground heat, flux [W/m**2]"
* Find suitable standard parameters names by:
  * Looking at similar datasets published at [PANGAEA](https://pangaea.de/)
  * Finding matching parameters in the [PANGAEA parameter database](https://www.pangaea.de/lists/parameter/all-byname)
* Look out for PANGAEA's special [geocodes](https://wiki.pangaea.de/wiki/Geocode) and their formatting

#### Download complete PANGAEA parameter list

In [41]:
### URL link to PANGAEA parameters
URL = "https://www.pangaea.de/lists/parameter/all-byname"
response = requests.get(URL)
### Save parameters as local file
open("PANGAEA_parameters.tab", "wb").write(response.content)

11273070

In [42]:
### Load parameter list
params = pd.read_csv('PANGAEA_parameters.tab', sep = '\t', on_bad_lines = "warn")

In [43]:
### Check out table size
print(f'There are currently {params.shape[0]} parameters available in PANGAEA')

There are currently 220941 parameters available in PANGAEA


In [44]:
### Short glance at the data table
pd.concat([params.head(4), params.tail(4)])

,Parameter,Abbreviation,Unit,ID parameter
0,-,NaN,NaN,16432
1,-,NaN,NaN,16431
2,"10,13-dimethyl-17-[2-(2-ethylcyclopropyl)ethan...","22,24-cyclocholest-5-ene/TOC",µg/g,191031
3,"10,13-dimethyl-17-[2-(2-ethylcyclopropyl)ethan...","22,24-cyclocholest-5-ene/sed",µg/g,191030
220937,"ν1 peak position, carbonate",ν1 peak pos CO3,1/cm,524691
220938,"ω_r, vorticity, cylindrical, radial component",ω_r,NaN,514819
220939,"ω_z, vorticity, cylindrical, axial component",ω_z,NaN,514821
220940,"ω_θ, vorticity, cylindrical, azimuthal component",ω_θ,NaN,514820


#### Lookup similar parameters in the existing list of PANGAEA parameters
Note: If you cannot find a suitable parameter we can create a new one for you

##### Find parameters containing your parameter name
Note: Also check for a matching unit for your parameter

In [45]:
### Find your parameter using partial string matching
params[params["Parameter"].str.contains("Latitude", case = False)]

,Parameter,Abbreviation,Unit,ID parameter
48569,Colatitude,Colat,deg,108668
103673,LATITUDE,Latitude,NaN,1600
103674,Latitude 2,Latitude 2,NaN,20716
103675,"Latitude, additional",Latitude+,NaN,31412
103676,"Latitude, center",Lat C,NaN,190123
103677,Latitude description,Latitude descr,NaN,160914
103678,"Latitude, difference",Latitude diff,NaN,517237
103679,Latitude (EPSG),Latitude EPSG,NaN,189957
103680,"Latitude, error",Latitude e,m,185820
103681,"Latitude, error",Latitude e,NaN,49734


In [46]:
### Find similar (not exact matches) PANGAEA parameters for your own parameter. This search is less sensitive to spelling mistakes
get_close_matches('latitude', params["Parameter"], n = 10)

['Colatitude',
 'Altitude',
 'Latitude 2',
 'Paleolatitude',
 'Platynite',
 'Magnitude',
 'Blattodea',
 'Amplitude',
 'Amplitude',
 'Amplitude']

#### Rename parameters

In [47]:
print('original column names: ',df.columns)

original column names:  Index(['Location', 'Threatened status', 'species', 'latitude (deg)',
       'lngitude (deg)', 'water depth (cm)', 'water temperature (degC)',
       'date/time', 'lat', 'lon'],
      dtype='object')


In [48]:
### define new column names 
new_col_names = ['Location []', 'IUCN Red List status []', 
                 'Species []', 'LATITUDE []', 'LONGITUDE []',
                 'DEPTH, water [m]', 'Temperature, water [°C]',
                 'DATE/TIME []', 'Latitude 2 []', 'Longitude 2 []']

In [49]:
### replace original column names with new column names
df.columns = new_col_names

print('new column names: ',df.columns)

new column names:  Index(['Location []', 'IUCN Red List status []', 'Species []', 'LATITUDE []',
       'LONGITUDE []', 'DEPTH, water [m]', 'Temperature, water [°C]',
       'DATE/TIME []', 'Latitude 2 []', 'Longitude 2 []'],
      dtype='object')


In [50]:
df.head(3)

,Location [],IUCN Red List status [],Species [],LATITUDE [],LONGITUDE [],"DEPTH, water [m]","Temperature, water [°C]",DATE/TIME [],Latitude 2 [],Longitude 2 []
1,"Bremerhaven, Germany",Near Threatened,Tursiops truncatus,69.3,11.5,NaN,0.995700,2014-07-24T23:00:00,53.549999,8.583333
2,"Bremerhaven, Germany",Endangered,Delphinus delphis,69.4,12.0,NaN,0.925061,2012-07-15T18:00:00,53.549999,8.583333
3,"Bremerhaven, Germany",Extinct in the Wild,Stenella coeruleoalba,69.5,12.5,2.5,0.570743,2013-03-03T02:00:00,53.549999,8.583333


In [51]:
### sort columns
df = df[['DATE/TIME []','LATITUDE []','LONGITUDE []',
         'Latitude 2 []', 'Longitude 2 []','DEPTH, water [m]',
         'Location []', 'Species []', 'IUCN Red List status []',
         'Temperature, water [°C]'
        ]]

In [52]:
df.head(3)

,DATE/TIME [],LATITUDE [],LONGITUDE [],Latitude 2 [],Longitude 2 [],"DEPTH, water [m]",Location [],Species [],IUCN Red List status [],"Temperature, water [°C]"
1,2014-07-24T23:00:00,69.3,11.5,53.549999,8.583333,NaN,"Bremerhaven, Germany",Tursiops truncatus,Near Threatened,0.995700
2,2012-07-15T18:00:00,69.4,12.0,53.549999,8.583333,NaN,"Bremerhaven, Germany",Delphinus delphis,Endangered,0.925061
3,2013-03-03T02:00:00,69.5,12.5,53.549999,8.583333,2.5,"Bremerhaven, Germany",Stenella coeruleoalba,Extinct in the Wild,0.570743


### 3.6. Conversions

#### Convert units
PANGAEA aims to reduce redundancy of units by converting units to one scale (if possible). During the lookup of parameters you can already see which unit scale is used in PANGAEA.

In [53]:
### DEPTH, water was initially in "cm", but in PANGAEA "m" is the most common unit used for this parameter
df['DEPTH, water [m]'] = df['DEPTH, water [m]']/100
df.head(4)

,DATE/TIME [],LATITUDE [],LONGITUDE [],Latitude 2 [],Longitude 2 [],"DEPTH, water [m]",Location [],Species [],IUCN Red List status [],"Temperature, water [°C]"
1,2014-07-24T23:00:00,69.3,11.5,53.549999,8.583333,NaN,"Bremerhaven, Germany",Tursiops truncatus,Near Threatened,0.995700
2,2012-07-15T18:00:00,69.4,12.0,53.549999,8.583333,NaN,"Bremerhaven, Germany",Delphinus delphis,Endangered,0.925061
3,2013-03-03T02:00:00,69.5,12.5,53.549999,8.583333,0.025,"Bremerhaven, Germany",Stenella coeruleoalba,Extinct in the Wild,0.570743
4,2012-02-10T09:00:00,69.6,13.0,53.549999,8.583333,0.030,"Bremerhaven, Germany",Tursiops truncatus,Least Concern,0.148282


### 3.7. URLs

In [54]:
# add URL in mew column
df['Uniform resource locator/link to reference []'] = "https://doi.org/10.1594/PANGAEA.945749"

In [55]:
df.head(2)

,DATE/TIME [],LATITUDE [],LONGITUDE [],Latitude 2 [],Longitude 2 [],"DEPTH, water [m]",Location [],Species [],IUCN Red List status [],"Temperature, water [°C]",Uniform resource locator/link to reference []
1,2014-07-24T23:00:00,69.3,11.5,53.549999,8.583333,NaN,"Bremerhaven, Germany",Tursiops truncatus,Near Threatened,0.995700,https://doi.org/10.1594/PANGAEA.945749
2,2012-07-15T18:00:00,69.4,12.0,53.549999,8.583333,NaN,"Bremerhaven, Germany",Delphinus delphis,Endangered,0.925061,https://doi.org/10.1594/PANGAEA.945749


In [56]:
### check if URL are working
[f'This link works' if requests.get(url).status_code == 200 else False for url in df['Uniform resource locator/link to reference []'][:2]] # check only the first 2 links

['This link works', 'This link works']

### 3.8. Event
An [EVENT](https://wiki.pangaea.de/wiki/Event) refers to the sampling event at which the data was obtained, such as a [station](https://www.pangaea.de/expeditions/) during a research expedition. An [EVENT](https://wiki.pangaea.de/wiki/Event) can also be described as metadata overview of a sampling device or sampling method

In [57]:
### add event column
df.insert(loc = 0, column = 'Event []', value = 'PS132_1-2')
df.head(2)

,Event [],DATE/TIME [],LATITUDE [],LONGITUDE [],Latitude 2 [],Longitude 2 [],"DEPTH, water [m]",Location [],Species [],IUCN Red List status [],"Temperature, water [°C]",Uniform resource locator/link to reference []
1,PS132_1-2,2014-07-24T23:00:00,69.3,11.5,53.549999,8.583333,NaN,"Bremerhaven, Germany",Tursiops truncatus,Near Threatened,0.995700,https://doi.org/10.1594/PANGAEA.945749
2,PS132_1-2,2012-07-15T18:00:00,69.4,12.0,53.549999,8.583333,NaN,"Bremerhaven, Germany",Delphinus delphis,Endangered,0.925061,https://doi.org/10.1594/PANGAEA.945749


## 4. Save curated data
* as tab-delimited text files
* Important: PANGAEA only accepts UTF-8 encoding to avoid unicode errors


In [58]:
### Save as tab-delimited txt file
df.to_csv('Curated_data.txt', index=False, sep="\t", encoding='utf-8')

## 5. Submit to PANGAEA
Congratulations! You finished curating your data, which was an important step forward to make your data more accessible, understandable und useable for future users.
Now you are ready to submit your data file(s) to [PANGAEA](https://www.pangaea.de/submit/).